In [1]:
import xlwings as xw
import pandas as pd
wb_cy = xw.Book('cytiva.xlsm')
SHEET_NAMES =  ['Temp_DB', 'Shipment information', '인수증', 
'대리점송장', '대리점 출고대기', '로컬리스트', 'In-Transit part report', '기타리스트',
 '출고리스트', 'Cytiva Inventory BIN']

In [2]:
ws_si = wb_cy.sheets[SHEET_NAMES[1]]

In [3]:
last_row = ws_si.range('A500000').end('up').row
last_col = ws_si.range('AAA9').end('left').column

In [4]:
last_row,last_col

(20496, 18)

In [5]:
rng_si = ws_si.range((10,1),(last_row,last_col))
rng_si
idx_si = ws_si.range((9,1),(9,last_col))

In [16]:
idx_si.value

['AWB#',
 'Trip#',
 'shipment no',
 'number of Packge',
 'Invoice Date',
 'ORDER NO',
 'Order Total',
 'Unit price(대리점)',
 'Ship to',
 'Arrival date',
 'Ship date/Receiving date',
 'POD \nReceipt \nDate',
 '유무상 여부',
 'remark',
 'Parcels number',
 '비고',
 'Status',
 'si_index']

In [7]:
rng_si.options(numbers=int).value[0]

[7764059026,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 'DHL특송건',
 datetime.datetime(2021, 1, 4, 0, 0),
 datetime.datetime(2021, 1, 6, 0, 0),
 None,
 '무상',
 None,
 None,
 None,
 None,
 36]

In [17]:
df_si =pd.DataFrame(rng_si.options(numbers=int).value,columns=idx_si.value)

In [19]:
## 출고하지않은 대리점 품목

df_si_br=df_si[df_si['remark']=='대리점']
df_br_not_shiped=df_si_br[pd.isna(df_si_br['Ship date/Receiving date'])&~(pd.isna(df_si_br['Arrival date']))]

In [20]:
#대리점에서 필요한 컬럼명
br_col = wb_cy.sheets['대리점송장'].range("A6:G6").value
br_col.append('Unit price(대리점)')
br_col.append('si_index')

In [21]:
df_br=df_br_not_shiped[br_col]
df_br['Unit price(대리점)'] = df_br[['Unit price(대리점)']].fillna(0)

C:\Users\lms46\AppData\Local\Temp\ipykernel_18944\1228965626.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_br['Unit price(대리점)'] = df_br[['Unit price(대리점)']].fillna(0)


In [22]:
df_br[df_br['Unit price(대리점)']==0]

,Arrival date,AWB#,Trip#,number of Packge,Parcels number,ORDER NO,Ship to,Unit price(대리점),si_index
20147,2022-10-25,DSV0322345,115384492,1,UPD3349097,2125207,동일과학,0,29608
20212,2022-10-27,STO0165477,115386556,1,UPD3348990,2125207,동일과학,0,29673
20213,2022-10-27,STO0165477,115386556,1,UPD3348976,2123806,동일과학,0,29674
20214,2022-10-27,STO0165477,115386556,None,UPD3348976,2125207,동일과학,0,29675
20361,2022-10-27,564073221450,115389345,1,PYD002917255,2125260,지더블유바이텍광주,0,29822
20380,2022-10-27,STO0165498,115389201,1,UPD3349708,2125246,동일과학,0,29841
20381,2022-10-27,STO0165498,115389201,1,UPD3350190,2125111,동일과학,0,29842
20382,2022-10-27,STO0165498,115389201,1,UPD3349726,2122883,동일과학,0,29843
20383,2022-10-27,STO0165498,115389201,1,UPD3349820,2125207,동일과학,0,29844
20415,2022-10-31,DSV0322341,115384491,1,UPD3349124,2124089,동일과학,0,29876


In [23]:
df_br['한화표시(\)'] = df_br['Unit price(대리점)']*1200

C:\Users\lms46\AppData\Local\Temp\ipykernel_18944\3636066061.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_br['한화표시(\)'] = df_br['Unit price(대리점)']*1200


In [36]:
df_br.set_index('Arrival date',inplace=True)

In [38]:
df_br.columns[:6]

Index(['AWB#', 'Trip#', 'number of Packge', 'Parcels number', 'ORDER NO',
       'Ship to'],
      dtype='object')

In [39]:
df_br[df_br.columns[:6]]

,AWB#,Trip#,number of Packge,Parcels number,ORDER NO,Ship to
Arrival date,,,,,,
2022-04-28,EWR0352882,110093392,1,PYD002818156,800588,바이오토프
2022-07-18,DSV0280973,114645173,1,UPD3304011,2123439,바이오토프
2022-07-19,STO0162933,114887327,1,UPD3305059,2123253,바이오토프
2022-07-19,STO0162933,114887333,1,UPD3305751,2123637,유비코리아
2022-07-19,STO0162958,114969331,1,UPD3306062,2123648,바이오토프
...,...,...,...,...,...,...
2022-10-31,STO0165597,115394022,1,UPD3352018,2125318,서린바이오사이언스
2022-10-31,STO0165597,115394022,1,UPD3351982,2125318,서린바이오사이언스
2022-10-31,STO0165597,115394022,1,UPD3351958,2125318,서린바이오사이언스


In [31]:
ws_br = wb_cy.sheets[SHEET_NAMES[3]]
ws_br

<Sheet [cytiva.xlsm]대리점송장>

In [41]:
ws_br.range('A6').value = df_br[df_br.columns[:6]]

In [43]:
df_br[df_br.columns[6:]].set_index('Unit price(대리점)')

,si_index,한화표시(\)
Unit price(대리점),,
523,24305,627600
624,26848,748800
402,26898,482400
1030,26927,1236000
168,26946,201600
...,...,...
437,29938,524400
1235,29939,1482000
335,29940,402000


In [44]:
df_br_2 = df_br[df_br.columns[6:]].set_index('Unit price(대리점)')

In [45]:
ws_br.range('J6').value = df_br_2

In [61]:
def get_df_rng(sheet=wb_cy.selection.sheet):
    """
    실제 값이 들어가있는 범위 구하기
    """
    sel_sht = sheet
    row_start_nm = sel_sht.range('A2').end('down').row
    row_end_nm = sel_sht.range('A1048576').end('up').row
    
    col_nm = sel_sht.range('XFD'+str(row_start_nm)).end('left').column
    
    return (row_start_nm,1),(row_end_nm,col_nm)
    

In [62]:
get_df_rng()

((6, 1), (76, 12))

In [73]:
br_rng = ws_br.range(get_df_rng()[0],get_df_rng()[1])

In [88]:
br_rng

<Range [cytiva.xlsm]대리점송장!$A$6:$L$76>

In [83]:
br_rng.font.size = 11
br_rng.font.name = '맑은 고딕'
br_rng.autofit()

In [87]:
br_rng.api.Borders.Weight = 2

## 아웃룩 파이썬 연동

In [94]:
import win32com.client as cli

In [160]:
outlook=win32com.client.Dispatch("Outlook.Application")
outlook_in = cli.Dispatch("Outlook.Application").GetNamespace("MAPI") # 아웃룩
inbox = outlook_in.GetDefaultFolder(6) # 받은편지함
msg = inbox.Items #메세지 정보
msg_counter = msg.count # 총갯수
print("메세지 갯수 : "+str(msg_counter))
 
for ms in msg:
    print("보낸사람 : "+str(ms.SenderName))

메세지 갯수 : 0


In [166]:
outlook_in.CurrentUser()

'lms46784678@gmail.com'

In [165]:
outlook.GetNamespace('MAPI').GetDefaultFolder(6)

<COMObject GetDefaultFolder>

In [151]:
outlook.GetDefaultFolder(6).Items

<COMObject <unknown>>

In [159]:
send_mail = outlook.CreateItem(0)

In [157]:
send_mail.To = "lms4678@naver.com" #메일 수신인
send_mail.Subject = "win32com Outlook 메일 전송 테스트 - 투손플레이스" #메일 제목
send_mail.HTMLBody = "test" #메일 내용(html)

In [158]:
send_mail.Save()

## 메일수신 IMAP

In [172]:
# !pip install imapclient
# !pip install pyzmail36

In [173]:
import imapclient
imap_obj = imapclient.IMAPClient('imap.gmail.com', ssl=True)

In [ ]:
imap_obj.login('email_address@gmail.com', 'MY_SECRET_PASSWORD')